<h1>PREDICTING RESTAURANT COST HACKANTHON PROJECT</h1>


In [1]:
# importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
#importing datasets 
#training set
data_train=pd.read_excel("Data_Train.xlsx")
#test set
data_test=pd.read_excel("Data_Test.xlsx")

In [3]:
data_train.shape

(12690, 9)

In [4]:
data_train.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [5]:
data_test.shape

(4231, 8)

In [6]:
data_test.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes


<h2>DATA EXPLORATION</h2>

In [7]:
#FEATURES OF TRAINING SET
print("columns:\n",data_train.columns)
print("\n no of rows: ",len(data_train),"\n no. of columns: ",len(data_train.columns))
print("\n data types:\n",data_train.dtypes)
print("\n contains null values: ",data_train.isnull().values.any())
print("\n no. of empty cells:\n",data_train.isnull().sum())
print("\n contains duplicate values: ",data_train.duplicated().values.any())
print("\n no. of duplicate values: ",data_train.duplicated().sum())

columns:
 Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES', 'COST'],
      dtype='object')

 no of rows:  12690 
 no. of columns:  9

 data types:
 TITLE            object
RESTAURANT_ID     int64
CUISINES         object
TIME             object
CITY             object
LOCALITY         object
RATING           object
VOTES            object
COST              int64
dtype: object

 contains null values:  True

 no. of empty cells:
 TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

 contains duplicate values:  True

 no. of duplicate values:  25


In [8]:
#FEATURES OF TRAINING SET
print("columns: ",data_test.columns)
print("\n no of rows: ",len(data_test),"\n no. of columns: ",len(data_test.columns))
print("\n data types:\n",data_test.dtypes)
print("\n contains null values: ",data_test.isnull().values.any())
print("\n no. of empty cells:\n",data_test.isnull().sum())
print("\n contains duplicate values: ",data_test.duplicated().values.any())
print("\n no. of duplicate values: ",data_test.duplicated().sum())

columns:  Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES'],
      dtype='object')

 no of rows:  4231 
 no. of columns:  8

 data types:
 TITLE            object
RESTAURANT_ID     int64
CUISINES         object
TIME             object
CITY             object
LOCALITY         object
RATING           object
VOTES            object
dtype: object

 contains null values:  True

 no. of empty cells:
 TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY              35
LOCALITY          30
RATING             2
VOTES            402
dtype: int64

 contains duplicate values:  True

 no. of duplicate values:  1


In [9]:
#merging training set and test set
merged_df=[data_train[['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY','RATING', 'VOTES']], data_test]
merged_df=pd.concat(merged_df)

In [10]:
#ANALYSING TITLES
titles=list(merged_df['TITLE'])
#finding max no. of titles in a single cell
max=1
for i in titles:
    if len(i.split(',')) > max:
        max=len(i.split(','))
print("maximum titles per cell: ",max)
#finding unique TITLES 
all_titles=[]
for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())
print("\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\nUnique Titles:\n", pd.Series(all_titles).unique())
all_titles = list(pd.Series(all_titles).unique())

maximum titles per cell:  2

Number of Unique Titles :  25

Unique Titles:
 ['CASUAL DINING' 'BAR' 'QUICK BITES' 'DESSERT PARLOR' 'CAFÉ'
 'MICROBREWERY' 'BEVERAGE SHOP' 'IRANI CAFE' 'BAKERY' 'NONE' 'PUB'
 'FINE DINING' 'SWEET SHOP' 'LOUNGE' 'FOOD COURT' 'FOOD TRUCK' 'MESS'
 'KIOSK' 'CLUB' 'CONFECTIONERY' 'DHABA' 'MEAT SHOP' 'COCKTAIL BAR'
 'PAAN SHOP' 'BHOJANALYA']


In [11]:
#ANALYSING CUISINES
cuisines=list(merged_df['CUISINES'])
#finding max no. of cuisines in a single cell
max=1
for i in cuisines:
    if len(i.split(',')) > max:
        max=len(i.split(','))
print("maximum cuisines per cell: ",max)
#finding unique CUISINES
all_cuisines = []
for i in cuisines :
    if len(i.split(',')) == 1:
         all_cuisines.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_cuisines.append(i.split(',')[it].strip().upper())
print("\nNumber of Unique Cuisines : ", len(pd.Series(all_cuisines).unique()))
print("\nUnique Cuisines:\n", pd.Series(all_cuisines).unique())
all_cuisines = list(pd.Series(all_cuisines).unique())

maximum cuisines per cell:  8

Number of Unique Cuisines :  130

Unique Cuisines:
 ['MALWANI' 'GOAN' 'NORTH INDIAN' 'ASIAN' 'MODERN INDIAN' 'JAPANESE'
 'CHINESE' 'BIRYANI' 'HYDERABADI' 'TIBETAN' 'DESSERTS' 'SEAFOOD' 'CAFE'
 'PIZZA' 'BURGER' 'BAR FOOD' 'SOUTH INDIAN' 'FAST FOOD' 'BEVERAGES'
 'ARABIAN' 'MUGHLAI' 'MAHARASHTRIAN' 'PARSI' 'THAI' 'BAKERY' 'MOMOS'
 'CONTINENTAL' 'EUROPEAN' 'ROLLS' 'ANDHRA' 'ITALIAN' 'BBQ' 'FINGER FOOD'
 'TEA' 'AMERICAN' 'HEALTHY FOOD' 'COFFEE' 'INDONESIAN' 'KOREAN' 'NEPALESE'
 'ICE CREAM' 'MEXICAN' 'KERALA' 'INDIAN' 'MITHAI' 'STREET FOOD'
 'MALAYSIAN' 'VIETNAMESE' 'IRANIAN' 'KEBAB' 'JUICES' 'SANDWICH'
 'MEDITERRANEAN' 'SALAD' 'GUJARATI' 'RAJASTHANI' 'TEX-MEX' 'ROAST CHICKEN'
 'BURMESE' 'CHETTINAD' 'NORTH EASTERN' 'LEBANESE' 'COFFEE AND TEA' 'GRILL'
 '' 'BIHARI' 'BENGALI' 'LUCKNOWI' 'AWADHI' 'STEAK' 'FRENCH' 'PORTUGUESE'
 'WRAPS' 'SRI LANKAN' 'ORIYA' 'ETHIOPIAN' 'KONKAN' 'SUSHI' 'SPANISH'
 'RUSSIAN' 'MANGALOREAN' 'TURKISH' 'BUBBLE TEA' 'AFGHAN' 'NAGA'
 'SINGAP

In [12]:
#ANALYSING CITY
all_cities=list(merged_df['CITY'])

#finding unique CITY
for i in range(len(all_cities)):
    if type(all_cities[i]) == float:
        all_cities[i] = 'NOT AVAILABLE'
    all_cities[i] = all_cities[i].strip().upper()     
print("\nNumber of Unique cities (WITHOUT INCLUSION): ", len(pd.Series(all_cities).unique()))
print("\nUnique Cities:\n", pd.Series(all_cities).unique())


Number of Unique cities (WITHOUT INCLUSION):  445

Unique Cities:
 ['THANE' 'CHENNAI' 'MUMBAI' 'BANGALORE' 'GURGAON' 'HYDERABAD' 'KOCHI'
 'THANE WEST' 'ANDHERI LOKHANDWALA' 'NEW DELHI' 'ANDHERI WEST'
 'MALAD EAST' '682036' 'BANGALOR' 'NAVI MUMBAI' 'BANDRA WEST' 'DELHI'
 'NOIDA' 'BANGALORE-560066' 'SECUNDERABAD' 'NOT AVAILABLE' 'INDIA'
 'MADHURANAGAR' 'CHENNAI TEYNAMPET' 'FARIDABAD' 'CHEMBUR.' 'MAHARASHTRA'
 'OPP GURUDWARA SHAKURPUR' 'TELAGANA LAND LINE:040-48507016' 'GHAZIABAD'
 'KARNATAKA' 'KERALA' 'EDAPPALLY' 'KADAVANTHRA' 'ERNAKULAM CIRCLE KOCHI'
 'BENGALORE' 'NEAR RELIANCE FRESH' 'KILPAUK' 'BENGALURU' 'KOTHAGUDA'
 'GOREGAON WEST' 'BANGLORE' 'TAMIL NADU' 'KAKKANAD' 'KOCHI ELAMKULAM'
 'OUTER RING ROAD' 'MULUND EAST'
 'SECUNDERABAD MAIN ROAD NEAR SIGNAL NMREC COLLEGE' 'TELANGANA'
 'PONNURUNI KOCHI' 'GACHIBOWLI' 'SEMMANCHERI'
 '5TH MAIN TEACHERS COLONY KORAMANGALA BLOCK 1 BANGALORE 560034'
 'MUMBAI MAHIM' 'POWAI (NEXT TO POWAI PLAZA)' 'DOMBIVALI EAST'
 'KOCHI VYTTILA' 'KANDIVALI' 'KOC

In [13]:
#ANALYSING LOCALITY
all_localities=list(merged_df['LOCALITY'])

#finding unique LOCALITY
for i in range(len(all_localities)):
    if type(all_localities[i]) == float:
        all_localities[i] = 'NOT AVAILABLE'
    all_localities[i] = all_localities[i].strip().upper()
print("\nNumber of Unique Localities (WITHOUT INCLUSION) : ", len(pd.Series(all_localities).unique()))
print("\nUnique Localities:\n", pd.Series(all_localities).unique())
all_localities = list(pd.Series(all_localities).unique())


Number of Unique Localities (WITHOUT INCLUSION) :  1611

Unique Localities:
 ['DOMBIVALI EAST' 'RAMAPURAM' 'SALIGRAMAM' ... 'OFF CARTER ROAD'
 'SRM BACK GATE' 'PERRY CROSS ROAD']


<h2>DATA CLEANING</h2>

In [14]:
#cleaning training set

#cleaning TITLE
titles=list(data_train['TITLE'])
# as there are max 2 titles per cell, we will split TITLE column into 2 parts
title1=[]
title2=[]
for i in titles:
    title1.append(i.split(',')[0].strip().upper())
    try :
        title2.append(i.split(',')[1].strip().upper())
    except :
        title2.append('NONE')
# appending NONE to Unique titles list
all_titles.append('NONE')

#cleaning CUISINES
cuisines=list(data_train['CUISINES'])
# as there are max 8 cuisines per cell, we will split CUISINES column into 8 parts
cuisine1=[]
cuisine2=[]
cuisine3=[]
cuisine4=[]
cuisine5=[]
cuisine6=[]
cuisine7=[]
cuisine8=[]
for i in cuisines:
    cuisine1.append(i.split(',')[0].strip().upper())
    try:
        cuisine2.append(i.split(',')[1].strip().upper())
    except:
        cuisine2.append('NONE')
    try:
        cuisine3.append(i.split(',')[2].strip().upper())
    except:
        cuisine3.append('NONE')
    try:
        cuisine4.append(i.split(',')[3].strip().upper())
    except:
        cuisine4.append('NONE')
    try:
        cuisine5.append(i.split(',')[4].strip().upper())
    except:
        cuisine5.append('NONE')
    try:
        cuisine6.append(i.split(',')[5].strip().upper())
    except:
        cuisine6.append('NONE')
    try:
        cuisine7.append(i.split(',')[6].strip().upper())
    except:
        cuisine7.append('NONE')
    try:
        cuisine8.append(i.split(',')[7].strip().upper())
    except:
        cuisine8.append('NONE')
#appending NONE to unique CUISINES list
all_cuisines.append('NONE')

#cleaning CITY
cities=list(data_train['CITY'])
for i in range(len(cities)):
    if type(cities[i]) == float:
        cities[i] = 'NOT AVAILABLE'
    cities[i] = cities[i].strip().upper()


#cleaning LOCALITY
localities=list(data_train['LOCALITY'])
for i in range(len(localities)):
    if type(localities[i]) == float:
        localities[i] = 'NOT AVAILABLE'
    localities[i] = localities[i].strip().upper()

#cleaning RATING
rates = list(data_train['RATING'])
for i in range(len(rates)) :
    try:
       rates[i] = float(rates[i])
    except :
       rates[i] = np.nan

# cleaning VOTES       
votes = list(data_train['VOTES'])
for i in range(len(votes)) :
    try:
       votes[i] = int(votes[i].split(" ")[0].strip())
    except :
       pass       

#creating new training set
data_train_new={}
data_train_new['TITLE1']=title1
data_train_new['TITLE2']=title2
data_train_new['RESTAURANT _ID']=data_train["RESTAURANT_ID"]
data_train_new['CUISINE1']=cuisine1
data_train_new['CUISINE2']=cuisine2
data_train_new['CUISINE3']=cuisine3
data_train_new['CUISINE4']=cuisine4
data_train_new['CUISINE5']=cuisine5
data_train_new['CUISINE6']=cuisine6
data_train_new['CUISINE7']=cuisine7
data_train_new['CUISINE8']=cuisine8
data_train_new['CITY'] = cities
data_train_new['LOCALITY'] = localities
data_train_new['RATING'] = rates
data_train_new['VOTES'] = votes
data_train_new['COST'] = data_train["COST"]

data_train_new=pd.DataFrame(data_train_new)
#printing new training set
data_train_new.head(5)

,TITLE1,TITLE2,RESTAURANT _ID,CUISINE1,CUISINE2,CUISINE3,CUISINE4,CUISINE5,CUISINE6,CUISINE7,CUISINE8,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,NONE,9438,MALWANI,GOAN,NORTH INDIAN,NONE,NONE,NONE,NONE,NONE,THANE,DOMBIVALI EAST,3.6,49.0,1200
1,CASUAL DINING,BAR,13198,ASIAN,MODERN INDIAN,JAPANESE,NONE,NONE,NONE,NONE,NONE,CHENNAI,RAMAPURAM,4.2,30.0,1500
2,CASUAL DINING,NONE,10915,NORTH INDIAN,CHINESE,BIRYANI,HYDERABADI,NONE,NONE,NONE,NONE,CHENNAI,SALIGRAMAM,3.8,221.0,800
3,QUICK BITES,NONE,6346,TIBETAN,CHINESE,NONE,NONE,NONE,NONE,NONE,NONE,MUMBAI,BANDRA WEST,4.1,24.0,800
4,DESSERT PARLOR,NONE,15387,DESSERTS,NONE,NONE,NONE,NONE,NONE,NONE,NONE,MUMBAI,LOWER PAREL,3.8,165.0,300


In [15]:
#cleaning test set

#cleaning TITLE
titles=list(data_test['TITLE'])
# as there are max 2 titles per cell, we will split TITLE column into 2 parts
title1=[]
title2=[]
for i in titles:
    title1.append(i.split(',')[0].strip().upper())
    try :
        title2.append(i.split(',')[1].strip().upper())
    except :
        title2.append('NONE')
        
#cleaning CUISINES
cuisines=list(data_test['CUISINES'])
# as there are max 8 cuisines per cell, we will split CUISINES column into 8 parts
cuisine1=[]
cuisine2=[]
cuisine3=[]
cuisine4=[]
cuisine5=[]
cuisine6=[]
cuisine7=[]
cuisine8=[]
for i in cuisines:
    cuisine1.append(i.split(',')[0].strip().upper())
    try:
        cuisine2.append(i.split(',')[1].strip().upper())
    except:
        cuisine2.append('NONE')
    try:
        cuisine3.append(i.split(',')[2].strip().upper())
    except:
        cuisine3.append('NONE')
    try:
        cuisine4.append(i.split(',')[3].strip().upper())
    except:
        cuisine4.append('NONE')
    try:
        cuisine5.append(i.split(',')[4].strip().upper())
    except:
        cuisine5.append('NONE')
    try:
        cuisine6.append(i.split(',')[5].strip().upper())
    except:
        cuisine6.append('NONE')
    try:
        cuisine7.append(i.split(',')[6].strip().upper())
    except:
        cuisine7.append('NONE')
    try:
        cuisine8.append(i.split(',')[7].strip().upper())
    except:
        cuisine8.append('NONE')

#cleaning CITY
cities=list(data_test['CITY'])
for i in range(len(cities)):
    if type(cities[i]) == float:
        cities[i] = 'NOT AVAILABLE'
    cities[i] = cities[i].strip().upper()


#cleaning LOCALITY
localities=list(data_test['LOCALITY'])
for i in range(len(localities)):
    if type(localities[i]) == float:
        localities[i] = 'NOT AVAILABLE'
    localities[i] = localities[i].strip().upper()

#cleaning RATING
rates = list(data_test['RATING'])
for i in range(len(rates)) :
    try:
       rates[i] = float(rates[i])
    except :
       rates[i] = np.nan

# cleaning VOTES       
votes = list(data_test['VOTES'])
for i in range(len(votes)) :
    try:
       votes[i] = int(votes[i].split(" ")[0].strip())
    except :
       pass       

data_test_new={}
data_test_new['TITLE1']=title1
data_test_new['TITLE2']=title2
data_test_new['RESTAURANT _ID']=data_test["RESTAURANT_ID"]
data_test_new['CUISINE1']=cuisine1
data_test_new['CUISINE2']=cuisine2
data_test_new['CUISINE3']=cuisine3
data_test_new['CUISINE4']=cuisine4
data_test_new['CUISINE5']=cuisine5
data_test_new['CUISINE6']=cuisine6
data_test_new['CUISINE7']=cuisine7
data_test_new['CUISINE8']=cuisine8
data_test_new['CITY'] = cities
data_test_new['LOCALITY'] = localities
data_test_new['RATING'] = rates
data_test_new['VOTES'] = votes

data_test_new=pd.DataFrame(data_test_new)
data_test_new.head(5)

,TITLE1,TITLE2,RESTAURANT _ID,CUISINE1,CUISINE2,CUISINE3,CUISINE4,CUISINE5,CUISINE6,CUISINE7,CUISINE8,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,NONE,4085,NORTH INDIAN,CHINESE,MUGHLAI,KEBAB,NONE,NONE,NONE,NONE,NOIDA,SECTOR 18,4.3,564.0
1,QUICK BITES,NONE,12680,SOUTH INDIAN,FAST FOOD,PIZZA,NORTH INDIAN,NONE,NONE,NONE,NONE,MUMBAI,GRANT ROAD,4.2,61.0
2,CASUAL DINING,NONE,1411,NORTH INDIAN,SEAFOOD,BIRYANI,CHINESE,NONE,NONE,NONE,NONE,MUMBAI,MARINE LINES,3.8,350.0
3,NONE,NONE,204,BIRYANI,NONE,NONE,NONE,NONE,NONE,NONE,NONE,FARIDABAD,NIT,3.8,1445.0
4,QUICK BITES,NONE,13453,SOUTH INDIAN,KERALA,NONE,NONE,NONE,NONE,NONE,NONE,KOCHI,KALOOR,3.6,23.0


<h2>DATA PRE-PROCESSING</h2>

In [16]:
#checking missing values in training set
print("empty cells: ",data_train_new.isnull().values.any())
print("no. of empty cells in each column:\n",data_train_new.isnull().sum())
# now we will replace na with 0
data_train_new.fillna(0,inplace=True)
#after filling 0
print("\nafter filling 0")
print("empty cells: ",data_train_new.isnull().values.any())
print("no. of empty cells in each column:\n",data_train_new.isnull().sum())

empty cells:  True
no. of empty cells in each column:
 TITLE1               0
TITLE2               0
RESTAURANT _ID       0
CUISINE1             0
CUISINE2             0
CUISINE3             0
CUISINE4             0
CUISINE5             0
CUISINE6             0
CUISINE7             0
CUISINE8             0
CITY                 0
LOCALITY             0
RATING            1204
VOTES             1204
COST                 0
dtype: int64

after filling 0
empty cells:  False
no. of empty cells in each column:
 TITLE1            0
TITLE2            0
RESTAURANT _ID    0
CUISINE1          0
CUISINE2          0
CUISINE3          0
CUISINE4          0
CUISINE5          0
CUISINE6          0
CUISINE7          0
CUISINE8          0
CITY              0
LOCALITY          0
RATING            0
VOTES             0
COST              0
dtype: int64


In [17]:
#checking missing values in test set
print("empty cells: ",data_test_new.isnull().values.any())
print("no. of empty cells in each column:\n",data_test_new.isnull().sum())
# now we will replace na with 0
data_test_new.fillna(0,inplace=True)
#after filling 0
print("\nafter filling 0")
print("empty cells: ",data_test_new.isnull().values.any())
print("no. of empty cells in each column:\n",data_test_new.isnull().sum())

empty cells:  True
no. of empty cells in each column:
 TITLE1              0
TITLE2              0
RESTAURANT _ID      0
CUISINE1            0
CUISINE2            0
CUISINE3            0
CUISINE4            0
CUISINE5            0
CUISINE6            0
CUISINE7            0
CUISINE8            0
CITY                0
LOCALITY            0
RATING            402
VOTES             402
dtype: int64

after filling 0
empty cells:  False
no. of empty cells in each column:
 TITLE1            0
TITLE2            0
RESTAURANT _ID    0
CUISINE1          0
CUISINE2          0
CUISINE3          0
CUISINE4          0
CUISINE5          0
CUISINE6          0
CUISINE7          0
CUISINE8          0
CITY              0
LOCALITY          0
RATING            0
VOTES             0
dtype: int64


In [18]:
#encoding categorical variable
le_titles = LabelEncoder()
le_cuisines = LabelEncoder()
le_city = LabelEncoder()
le_locality = LabelEncoder()

le_titles.fit(all_titles)
le_cuisines.fit(all_cuisines)
le_city.fit(all_cities)
le_locality.fit(all_localities)

#training set
data_train_new['TITLE1']=le_titles.transform(data_train_new['TITLE1'])
data_train_new['TITLE2']=le_titles.transform(data_train_new['TITLE2'])
data_train_new['CUISINE1']=le_cuisines.transform(data_train_new['CUISINE1'])
data_train_new['CUISINE2']=le_cuisines.transform(data_train_new['CUISINE2'])
data_train_new['CUISINE3']=le_cuisines.transform(data_train_new['CUISINE3'])
data_train_new['CUISINE4']=le_cuisines.transform(data_train_new['CUISINE4'])
data_train_new['CUISINE5']=le_cuisines.transform(data_train_new['CUISINE5'])
data_train_new['CUISINE6']=le_cuisines.transform(data_train_new['CUISINE6'])
data_train_new['CUISINE7']=le_cuisines.transform(data_train_new['CUISINE7'])
data_train_new['CUISINE8']=le_cuisines.transform(data_train_new['CUISINE8'])
data_train_new['CITY']=le_city.transform(data_train_new['CITY'])
data_train_new['LOCALITY']=le_locality.transform(data_train_new['LOCALITY'])

#test set
data_test_new['TITLE1']=le_titles.transform(data_test_new['TITLE1'])
data_test_new['TITLE2']=le_titles.transform(data_test_new['TITLE2'])
data_test_new['CUISINE1']=le_cuisines.transform(data_test_new['CUISINE1'])
data_test_new['CUISINE2']=le_cuisines.transform(data_test_new['CUISINE2'])
data_test_new['CUISINE3']=le_cuisines.transform(data_test_new['CUISINE3'])
data_test_new['CUISINE4']=le_cuisines.transform(data_test_new['CUISINE4'])
data_test_new['CUISINE5']=le_cuisines.transform(data_test_new['CUISINE5'])
data_test_new['CUISINE6']=le_cuisines.transform(data_test_new['CUISINE6'])
data_test_new['CUISINE7']=le_cuisines.transform(data_test_new['CUISINE7'])
data_test_new['CUISINE8']=le_cuisines.transform(data_test_new['CUISINE8'])
data_test_new['CITY']=le_city.transform(data_test_new['CITY'])
data_test_new['LOCALITY']=le_locality.transform(data_test_new['LOCALITY'])

In [19]:
#dependent variable for training set
#as we have to predict cost, we have selected cost colun which is at last position
y_train=data_train_new.iloc[:,-1].values
y_train

array([1200, 1500,  800, ..., 1300,  400,  600], dtype=int64)

In [20]:
#dependent variables for training set
x_train=data_train_new.iloc[:,0:-1].values
x_train

array([[5.0000e+00, 2.0000e+01, 9.4380e+03, ..., 3.2000e+02, 3.6000e+00,
        4.9000e+01],
       [5.0000e+00, 1.0000e+00, 1.3198e+04, ..., 1.2040e+03, 4.2000e+00,
        3.0000e+01],
       [5.0000e+00, 2.0000e+01, 1.0915e+04, ..., 1.2610e+03, 3.8000e+00,
        2.2100e+02],
       ...,
       [1.6000e+01, 2.0000e+01, 1.1133e+04, ..., 1.5430e+03, 4.0000e+00,
        6.0800e+02],
       [5.0000e+00, 2.0000e+01, 6.1340e+03, ..., 7.0600e+02, 3.5000e+00,
        3.2000e+01],
       [5.0000e+00, 2.0000e+01, 6.4300e+03, ..., 1.5340e+03, 3.4000e+00,
        6.7700e+02]])

In [21]:
#independent variables for test set
x_test=data_test_new.iloc[:,:].values
x_test

array([[5.000e+00, 2.000e+01, 4.085e+03, ..., 1.307e+03, 4.300e+00,
        5.640e+02],
       [2.300e+01, 2.000e+01, 1.268e+04, ..., 4.170e+02, 4.200e+00,
        6.100e+01],
       [5.000e+00, 2.000e+01, 1.411e+03, ..., 7.590e+02, 3.800e+00,
        3.500e+02],
       ...,
       [2.300e+01, 2.000e+01, 8.617e+03, ..., 1.364e+03, 3.700e+00,
        5.300e+01],
       [2.300e+01, 2.000e+01, 6.485e+03, ..., 6.290e+02, 0.000e+00,
        0.000e+00],
       [2.300e+01, 2.000e+01, 2.421e+03, ..., 1.136e+03, 3.700e+00,
        6.300e+01]])

In [22]:
#feature scaling
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)
# as y_train is in 1-D array, we will first convert it into 2-D array
y_train=y_train.reshape(len(y_train),1)
y_train=sc.fit_transform(y_train)
y_train=y_train.ravel()

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
#now we will predict the COST
gbr=GradientBoostingRegressor( loss = 'huber',learning_rate=0.001,n_estimators=400, max_depth=6
                              ,subsample=1, verbose=False,random_state=126) 
gbr.fit(x_train,y_train)
y_pred=sc.inverse_transform(gbr.predict(x_test))
y_pred

array([649.15165903, 439.99866985, 540.0913419 , ..., 440.54669218,
       452.72080545, 442.15372983])

In [24]:
#converting to dataframe
y_pred=pd.DataFrame(y_pred,columns=['COST'])
y_pred

,COST
0,649.151659
1,439.998670
2,540.091342
3,487.854667
4,421.450149
5,485.325921
6,523.447669
7,620.888789
8,480.080263
9,522.870415


In [28]:
#saving the output to excel file
y_pred.to_excel("cost.xlsx",index=False)